In [ ]:
import cv2
import numpy as np

# Function to calculate the angle between three points
def calculate_angle(pointA, pointB, pointC):
    if pointA is None or pointB is None or pointC is None:
        return None
    vecAB = np.array(pointB) - np.array(pointA)
    vecBC = np.array(pointC) - np.array(pointB)
    cosine_angle = np.dot(vecAB, vecBC) / (np.linalg.norm(vecAB) * np.linalg.norm(vecBC))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Load pose estimation model
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

# Process video
video_path = "input.mp4"  # Path to the input video
cap = cv2.VideoCapture(video_path)
output_path = "output.mp4"

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# VideoWriter to save the output
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]
threshold = 0.1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Prepare frame for pose estimation
    blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(blob)
    output = net.forward()

    height, width, _ = frame.shape
    scaleX = width / output.shape[3]
    scaleY = height / output.shape[2]

    # Detect keypoints
    points = []
    for i in range(nPoints):
        probMap = output[0, i, :, :]
        _, prob, _, point = cv2.minMaxLoc(probMap)
        x = scaleX * point[0]
        y = scaleY * point[1]
        if prob > threshold:
            points.append((int(x), int(y)))
        else:
            points.append(None)

    # Draw skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (255, 255, 0), 2)
            cv2.circle(frame, points[partA], 8, (255, 0, 0), thickness=-1, lineType=cv2.FILLED)

    # Pose classification
    left_angle = calculate_angle(points[8], points[9], points[10])  # Left leg
    right_angle = calculate_angle(points[11], points[12], points[13])  # Right leg

    if left_angle and right_angle:
        avg_angle = (left_angle + right_angle) / 2
        if avg_angle < 120:
            pose = "Sitting"
        else:
            pose = "Standing"
    else:
        pose = "Unable to classify"

    # Overlay pose classification on the frame
    cv2.putText(frame, f"Pose: {pose}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame (optional, for live preview)
    cv2.imshow("Pose Estimation", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()